In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import gc
# import nltk
# nltk.download('stopwords')
import math
import pickle
import os
from smart_open import open
# from nltk.corpus import stopwords
import sklearn
from sklearn.metrics import classification_report
from keras.layers import Dropout
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# from gensim.models.keyedvectors import KeyedVectors

# import tensorflow as tf

# from tensorflow import keras

# from tensorflow.keras.layers import LSTM, Dense, Bidirectional
# from tensorflow.keras import layers


# from tensorflow.keras.utils import plot_model
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input
# from tensorflow.keras.layers import concatenate


In [ ]:
dev_location = "dev_data"
test_location = "test_data"
train_location = "train_data"

devData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
testData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
trainData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]


dataset = np.concatenate((devData, np.concatenate((testData, trainData))))

gc.collect()
max_num_words = 17
# model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/MCA Dataset/Copy of GoogleNews-vectors-negative300.bin.gz', binary=True)
# stop_words = set(stopwords.words('english'))


In [ ]:
def getAudioData(patientID, location, textD):
    fileName = "/content/drive/My Drive/MCA Dataset/" + str(location) + "/" + str(int(patientID)) + "_processed.csv"  # Adjust the file naming convention
    data = pd.read_csv(fileName)  # No header as columns are now known

    # If your CSV has headers, you might not need to use `.values` and instead work with DataFrame directly
    data_values = data.values if hasattr(data, 'values') else data

    # Apply the helper function (if it's still needed)
    # data_processed = audioDataHelper(data_values)

    sentenceDatas = []
    for sentence in textD:
        sentenceStartime = sentence[0]
        sentenceEndTime = sentence[1]
        startIndex = math.floor(sentenceStartime / 0.01)  # Assuming your data is still sampled at 100Hz
        endIndex = math.ceil(sentenceEndTime / 0.01)

        sentenceData = data_values[startIndex: endIndex]
        if sentenceData.size == 0:
            # print(f"Skipping empty slice at index range: {startIndex}-{endIndex}")
            continue  # Skip the current segment and continue with the next


        sentenceData = np.average(sentenceData, axis=0)  # Averaging over time
        sentenceData = sentenceData.reshape(1, -1)  # Reshaping to ensure it's a 2D array
        sentenceDatas.append(sentenceData)

    sentenceDatas = np.array(sentenceDatas)

    # If you need to reshape it into a specific form, do it here
    sentenceDatas = np.reshape(sentenceDatas, (textD.shape[0], -1))
    return sentenceDatas


In [ ]:
def checkDataPointExistence(patientID, split):
  for i in split:
    if(patientID == i[0]):
      return True
  return False

def getData(patientID, location,textD):
  # print("PatientID: " + str(int(patientID)))
  retData = [int(patientID)]
  # textD = getTextData(patientID, location)
  audioD = getAudioData(patientID, location, textD)
  # videoD = getVideoData(str(int(patientID)), location, textD)
  # patientD = np.concatenate((textD, audioD, videoD), axis = 1)
  # print("Final Patient Data: " + str(patientD.shape))
  return audioD

  # return textD,audioD,videoD

# def getTextData(patientID, location):
#   fileName = "/content/drive/My Drive/MCA Dataset/"+ str(location) + "/" + str(int(patientID)) + "_TRANSCRIPT.csv"
#   file = np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))

#   # Remove All Utterences By Ellie:
#   for i in range(len(file)):
#     if(file[i][2] != 'Participant'):
#       np.delete(file, i)
#       i-=1

#   # Remove Speaker Columnn
#   file = np.delete(file, 2, 1)

#   # Convert Text Into Word Vectors:
#   w2vs = np.zeros((1, max_num_words*300))
#   for i in range(len(file)):
#     sentence = file[i][2]
#     w2v = returnWordToVec(sentence)
#     w2vs = np.concatenate((w2vs, w2v), axis = 0)
#   w2vs = np.delete(w2vs, 0, 0)

#   # Delete Sentences and Replace With W2Vs
#   file = np.delete(file, 2, 1)
#   # print(file)
#   file = np.concatenate((file, w2vs), axis = 1)
#   return file

# def remove_StopWords(sentence):
#     filtered_sentence = []
#     for w in sentence:
#         if w not in stop_words:
#             filtered_sentence.append(w)
#     return filtered_sentence

# def returnWordToVec(sentence):
#   global max_num_words, stop_words, model
#   sentence = str(sentence).split(" ")
#   sentence = remove_StopWords(sentence)
#   index_word = 0
#   wordMatrix = np.zeros(max_num_words*300)
#   for j in range(min(max_num_words, len(sentence))):
#     try:
#       word = sentence[j]
#       if(word[0] == '<'):
#         if(word.find('>')!=-1):
#           word = word[1:-1]
#         else:
#           word = word[1:]
#       else:
#         if(word.find('>')!=-1):
#           word = word[0:-1]
#       ss = np.array(model[word])
#       wordMatrix[index_word*300:(index_word+1)*300] = ss
#       index_word+=1
#     except Exception as e:
#       continue
#   wordMatrix = np.array(wordMatrix.reshape(1,-1))
#   return wordMatrix

def audioDataHelper(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    X = np.array(X)
    return X




In [ ]:
audio_train_final = 'audio_train_final.pkl'
text_train_final = 'text_train_final.pkl'
Ytrain_final = 'Ytrain.pkl'

audio_test_final = 'audio_test_final.pkl'
text_test_final = 'text_test_final.pkl'
Ytest_final = 'Ytest.pkl'

audio_dev_final = 'audio_dev_final.pkl'
text_dev_final = 'text_dev_final.pkl'
Ydev_final = 'Ydev.pkl'

In [ ]:
# Function to load progress if file exists, else return an empty list
import os
def load_progress(file_name):
    if os.path.exists(file_name):
        with open(file_name, 'rb') as f:
            return pickle.load(f)
    else:
        return []

In [ ]:
vector_test = load_progress('vectors_train.pkl')

In [ ]:
audio_train = load_progress(audio_train_final)
text_train = load_progress(text_train_final)
Ytrain = load_progress(Ytrain_final)

# audio_test = load_progress(audio_test_final)
# text_test = load_progress(text_test_final)
# Ytest = load_progress(Ytest_final)


# Check if lists are empty, then initialize them
if not audio_train: audio_train = []
# if not audio_test: audio_test = []
if not text_train: text_train = []
if not Ytrain: Ytrain = []
# if not Ytest: Ytest = []
# if not text_test: text_test = []



In [ ]:
# audio_dev = []
# text_dev =[]
# Ydev = []

In [ ]:
a=0


for datapoint in dataset:

    if checkDataPointExistence(datapoint[0], trainData):
        # Process test data
        text = vector_test[a]
        audio = getData(datapoint[0], train_location,text)
        audio_train.append(audio)
        text_train.append(text)
        Ytrain.append(datapoint[1])
        a+=1
        print(a)

# print(a)

# # Save progress after processing test data
with open(audio_train_final, 'wb') as f:
    pickle.dump(audio_train, f)
with open(text_train_final, 'wb') as f:
    pickle.dump(text_train, f)
with open(Ytrain_final, 'wb') as f:
    pickle.dump(Ytrain, f)

# print(text1 == text2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107


In [ ]:
a=0
for datapoint in dataset:


    if checkDataPointExistence(datapoint[0], trainData):
        # Process train data
        text, audio = getData(datapoint[0], train_location)
        audio_train.append(audio)
        text_train.append(text)
        Ytrain.append(datapoint[1])
        a+=1

print(a)

# Save progress after processing train data
with open(audio_train_final, 'wb') as f:
    pickle.dump(audio_train, f)
with open(text_train_final, 'wb') as f:
    pickle.dump(text_train, f)
with open(Ytrain_final, 'wb') as f:
    pickle.dump(Ytrain, f)

In [ ]:
# Load the data from the first pickle file
import pickle
with open('Ydev.pkl', 'rb') as file:
    text_train = pickle.load(file)

# Load the data from the second pickle file
with open('Ytrain.pkl', 'rb') as file:
    text_train2 = pickle.load(file)

# Combine the data (This assumes that the data is list-like, adjust as necessary for your data structure)
combined_data = text_train + text_train2

# Save the combined data to a new pickle file
with open('Ytrain_X.pkl', 'wb') as file:
    pickle.dump(combined_data, file)

print("Data combined and saved successfully.")


Data combined and saved successfully.


In [ ]:
audio = load_progress('audio_train_finalX.pkl')
text = load_progress('text_train_finalX.pkl')
Y = load_progress('Ytrain_X.pkl')

In [ ]:
print(len(audio),len(text),len(Y))

142 142 142
